Read sample table from Terra workspace

In [ ]:
import dalmatian
from dataclasses import dataclass
import pandas as pd


namespace = "broad-firecloud-ccle"
workspaces = ["DepMap_WES_CN_hg38", "DepMap_WGS_CN"]
dest_dataset = "depmap-omics.maf_staging_0916"


In [ ]:

@dataclass
class Transfer:
    srcs : str
    dest_table : str
    cds_id : str

def get_transfers(workspace):
    wm = dalmatian.WorkspaceManager(f"{namespace}/{workspace}")

    sample = wm.get_entities("sample")
    sample = sample.reset_index()

    transfers = []
    for rec in sample.to_dict("records"):
        if isinstance(rec['full_file'], list):
            dest_table = f"{dest_dataset}.stage_maf_{rec['sample_id'].replace('-', '_')}"
            transfers.append(Transfer(rec['full_file'], dest_table, rec["sample_id"]))

    return transfers

transfers = []
for workspace in workspaces:
    transfers.extend(get_transfers(workspace))

In [ ]:
len(transfers)

Create "external" tables, one per cds_id from the associated uris

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
import re

from google.api_core.exceptions import Conflict

def create_ext_table(srcs, dest_table, job_prefix):
    table = bigquery.Table(dest_table)

    external_config = bigquery.ExternalConfig(bigquery.external_config.ExternalSourceFormat.PARQUET)
    external_config.source_uris = srcs
    table.external_data_configuration = external_config

    client.create_table(table, exists_ok=True)


In [ ]:
from tqdm.notebook import tqdm

def cleanup_uris(uris):
    result = []
    for uri in uris:
        result.extend([x.strip() for x in uri.split(",")])
    return list(set(result))

#transfers = transfers[:10]

# create a bunch of tables which correspond to CDS_IDs, because we want to add CDS_ID as a column
for transfer in tqdm(transfers):
    uris = cleanup_uris(transfer.srcs) # at least one row has a entry which looks like a string containing a comma seperated list instead of a real list
    create_ext_table(uris, transfer.dest_table, "t2")


In [ ]:
table = client.get_table(table_id)

In [ ]:
transfer

Copy from the external table into a single table adding the cds_id to each row.

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd

def create_batches(data, batch_size):
    return [data[x:x+batch_size] for x in range(0, len(data), batch_size)]

# Concatenate table adding CDS_ID to the dest table
def concatenate_tables(dest_table, transfers, job_prefix, parallelism):
    create_table_stmt = f"create table if not exists {dest_table} as select 'invalid' CDS_ID, * from `{transfers[0].dest_table}` limit 0"
    job = client.query(create_table_stmt)
    job.result() # wait for completion

    # figure out which cds_ids have already been loaded
    already_loaded = set(pd.read_gbq(f"""select distinct cds_id from `{dest_dataset}.merged_maf` """)["cds_id"])
    
    # drop transfers already loaded
    remaining_transfers = [x for x in transfers if x.cds_id not in already_loaded]
    print(f"{len(already_loaded)} CDS IDs already loaded. {len(remaining_transfers)} of {len(transfers)} tables need to be loaded")
    transfers = remaining_transfers
          
    batches = create_batches(transfers, parallelism)
    #return
    
    for batch in tqdm(batches, desc="batch", position=0):
        jobs = []

        # submit a batch to run in parallel
        for transfer in tqdm(batch, desc=" submit", position=1, leave=False):
            append_stmt = f"insert into {dest_table} select '{transfer.cds_id}' CDS_ID, * from {transfer.dest_table} where hugo_symbol != '' and hugo_symbol is not NULL"

            job = client.query(append_stmt)            
            jobs.append(job)
        
        # wait for batch to complete
        for job in tqdm(jobs, desc=" wait", position=1, leave=False):            
            job.result() # wait for completion
        

In [ ]:
concatenate_tables(f"{dest_dataset}.merged_maf", transfers, "t5", parallelism=10)

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
q = client.query(f"""select * from (
  SELECT chrom, pos, variant_type, count(1) mut_count FROM `{dest_dataset}.merged_maf` 
  where hugo_symbol='BRAF'
  group by chrom, pos, variant_type) where mut_count > 10
  LIMIT 1000""")


In [ ]:
result = q.result()

In [ ]:
pd.DataFrame(result)

In [ ]:
result

In [ ]:
import pandas as pd
df = pd.read_gbq(f"""select * from (
  SELECT chrom, pos, variant_type, count(1) mut_count FROM `{dest_dataset}.merged_maf` 
  group by chrom, pos, variant_type)
  LIMIT 1000""")

In [ ]:
df

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()
import re

from google.api_core.exceptions import Conflict



In [ ]:
tables = list(client.list_tables("depmap-omics.maf_staging_0916"))

In [ ]:
stage_mafs = [x.table_id[len("stage_maf_"):].replace("_", "-") for x in tables if x.table_id.startswith("stage_maf_CDS")]

In [ ]:
import pandas as pd
df = pd.read_gbq(f"""select distinct cds_id from `{dest_dataset}.merged_maf` """)

In [ ]:
set(stage_mafs).difference(set(df["cds_id"]))

In [ ]:
jobs = list(client.list_jobs())

In [ ]:
missing = [x for x in jobs if "CDS_dhBHhw" in x.job_id]

In [ ]:
missing[0].errors

In [ ]:
missing

In [ ]:
just_one_transfer = [x for x in transfers if "CDS_dhBHhw" in x.dest_table]


In [ ]:
concatenate_tables(f"{dest_dataset}.merged_maf", just_one_transfer, "t6", parallelism=10)

In [ ]:
job

In [ ]:
missing[0].result()

In [ ]:
transfers[0]